# User Guide

In [2]:
import ipcoal
import toytree

## Workflow

The user interacts with ipcoal through the Model object, through which the following are defined:
* Tree topology
* Effective population size
* Admixture
* Substitution rate
* Recombination rate

Then, the methods `.sim_loci()` or `.sim_snps()` generate data based on the input parameters.

The resulting simulated data consists of a dataframe object that holds the simulated genealogies, and matrices that contain the actual simulated sequences.

After data simulation with `.sim_loci`, ipcoal can automatically infer a gene tree from the simulated data at each locus. These gene trees are then incorporated into the dataframe object.

## Defining the tree topology.

`ipcoal` uses a `toytree` tree object as input for the toplogy of the species tree. You can read more about toytree at [the toytree docs](https://toytree.readthedocs.io/en/latest/index.html).

A `toytree` tree object can be generated three different ways:

#### random topology

In [23]:
# define the tree topology
tre = toytree.rtree.coaltree(5)
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t175a3e629648497d950a1ad2d4a327ce" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 0.1199
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 0.1199
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 0.1223
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.0680
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.0680
support: 100
height: 0.0000 4 idx: 5
name: 5
dist: 0.0024
support: 100
height: 0.1199 5 idx: 6
name: 6
dist: 0.0768
support: 100
height: 0.1223 6 idx: 7
name: 7
dist: 0.1312
support: 100
height: 0.0680 7 idx: 8
name: 8
dist: 0.0001
support: 100
height: 0.1991 8 0.00 0.10 0.20

Using the coaltree function multiple times can give different topologies and different node heights:

In [25]:
# generate 3 randome trees, save as a multitree object
mtre = toytree.mtree([toytree.rtree.coaltree(5) for i in range(3)])
# plot the trees
mtre.draw_tree_grid(
    start=0, 
    nrows=1, ncols=3, 
    shared_axis=True,
    edge_type='c',
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t80dcabae785e4935807a1329b790bd65" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 525.0 250.0" width="525.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 0.0 0.2 0.5

Node heights can be randomly shifted using the `node_slider()` method:

In [29]:
# apply `node_slider()` three times to the first tree in our multitree object
nodeslide_trees = toytree.mtree([mtre.treelist[0].mod.node_slider() for i in range(3)])
# plot the nodeslider trees:
nodeslide_trees.draw_tree_grid(
    start=0, 
    nrows=1, ncols=3, 
    shared_axis=True,
    edge_type='c',
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="td341e9be1c2b4a769456d0ed0793422c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 525.0 250.0" width="525.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 0.0 0.2 0.3

#### balanced/imbalanced topology

In [31]:
# define a 6-tip balanced tree
tre = toytree.rtree.baltree(6)
tre.draw(tree_style='c');
# define a 6-tip imbalanced tree
tre = toytree.rtree.imbtree(6)
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t7fc0b778b98847cca1233b0b4a7c789c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 0.3333
support: 100
height: 0.0000 0 idx: 1
name: r2
dist: 0.3333
support: 100
height: 0.0000 1 idx: 2
name: r0
dist: 0.6667
support: 100
height: 0.0000 2 idx: 3
name: r4
dist: 0.3333
support: 100
height: 0.0000 3 idx: 4
name: r5
dist: 0.3333
support: 100
height: 0.0000 4 idx: 5
name: r3
dist: 0.6667
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 0.3333
support: 100
height: 0.3333 6 idx: 7
name: 7
dist: 0.3333
support: 100
height: 0.3333 7 idx: 8
name: 8
dist: 0.3333
support: 100
height: 0.6667 8 idx: 9
name: 9
dist: 0.3333
support: 100
height: 0.6667 9 idx: 10
name: 10
dist: 0.3333
support: 100
height: 1.0000 10 0.0 0.5 1.0

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t53ed5f97614745ffa81bf4370ca05d2e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 0.2000
support: 100
height: 0.0000 0 idx: 1
name: r0
dist: 0.2000
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 0.4000
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.6000
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.8000
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 1.0000
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 0.2000
support: 100
height: 0.2000 6 idx: 7
name: 7
dist: 0.2000
support: 100
height: 0.4000 7 idx: 8
name: 8
dist: 0.2000
support: 100
height: 0.6000 8 idx: 9
name: 9
dist: 0.2000
support: 100
height: 0.8000 9 idx: 10
name: 10
dist: 0.2000
support: 100
height: 1.0000 10 0.0 0.5 1.0

#### topology from newick

In [55]:
tre = toytree.tree(
             '((r5:2.62256,(r4:0.96563,r3:0.96563)100:1.65693)100:\
              9.63792,(r2:7.42816,(r1:4.0738,r0:4.0738)100:\
              3.35436)100:4.83232);'
            )
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t0b01caa97c2b41d7abc880f6b8c028cb" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 4.0738
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 4.0738
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 7.4282
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.9656
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.9656
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 2.6226
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 3.3544
support: 100
height: 4.0738 6 idx: 7
name: 7
dist: 1.6569
support: 100
height: 0.9656 7 idx: 8
name: 8
dist: 4.8323
support: 100
height: 7.4282 8 idx: 9
name: 9
dist: 9.6379
support: 100
height: 2.6226 9 idx: 10
name: 10
dist: 1.0000
support: 100
height: 12.2605 10 0 6 12

#### Rescaling tree height

`ipcoal` interprets node heights as units of generations, so on phylogenetic scales we expect the node heights to be of pretty large magnitudes.

In [58]:
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6a9287cc62bd4347ac59f6eb2112f4bf" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 4.0738
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 4.0738
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 7.4282
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.9656
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.9656
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 2.6226
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 3.3544
support: 100
height: 4.0738 6 idx: 7
name: 7
dist: 1.6569
support: 100
height: 0.9656 7 idx: 8
name: 8
dist: 4.8323
support: 100
height: 7.4282 8 idx: 9
name: 9
dist: 9.6379
support: 100
height: 2.6226 9 idx: 10
name: 10
dist: 1.0000
support: 100
height: 12.2605 10 0 6 12

In [63]:
rescale_tre = tre.mod.node_scale_root_height(treeheight=1e6)
rescale_tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6bc49da2ba7b4b43bc1af8e1eae6c059" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 332270.8409
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 332270.8409
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 605862.0870
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 78759.5592
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 78759.5592
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 213903.5340
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 273591.2460
support: 100
height: 332270.8409 6 idx: 7
name: 7
dist: 135143.9748
support: 100
height: 78759.5592 7 idx: 8
name: 8
dist: 394137.9130
support: 100
height: 605862.0870 8 idx: 9
name: 9
dist: 786096.4660
support: 100
height: 213903.5340 9 idx: 10
name: 10
dist: 81562.8752
support: 100
height: 1000000.0000 10 0 500000 1000000

## Defining the demographic model.

The `ipcoal` Model object allows users to designate effective population size, admixture, substitution rates, and recombination rates.

### Effective population size (Ne)

#### One Ne for the whole tree

#### Branch-specific Ne

### Admixture

### Subsitution and recombination

## Simulating unlinked SNPs

In [ ]:
# simulate N unlinked SNPs (will run until N snps are produced)
model.sim_snps(1000)

In [ ]:
model.df

## Simulating loci

In [ ]:
# simulate N loci of len L 
model.sim_loci(100, 3000)

In [ ]:
# view the genealogies and stats in a table
model.df

In [ ]:
# view sequence data as an array
model.seqs

## Inferring gene trees

* For use with multilocus data.

Once multilocus data has been simulated, the `.infer_gene_trees()` method automates the gene tree inference process across all loci.

In [ ]:
# define the tree topology
tre = toytree.rtree.unittree(5, treeheight=1e6, seed=111)
# define 
model = ipcoal.Model(tree=tre, Ne=1e6, recomb=1e-9, seed=111)
# infer a tree for every locus
model.infer_gene_trees(inference_method='raxml')

Alternative inference methods include `'mrbayes'` and `'iqtree'`.

After running gene tree inference, our inferred gene trees can be found in the new `inferred_tree` column of our dataframe:

model.df.head(10)

## Plotting

## Writing to disk

In [ ]:
# save table to a CSV file
model.df.to_csv("./tree_table.csv")

# write loci as separate phylip files to a directory
model.write_loci_to_phylip(outdir="./tests")

# write concatenated loci or snps to a phylip file
model.write_concat_to_phylip(outdir="./tests", name="test.phy")